In [1]:
import pandas as pd
import numpy as np
import math
from scipy.stats import norm
import matplotlib.pyplot as plt

In [137]:
#Construit des matrices de transition projetées (get_projection())
#A partir :
#     - d'une matrice de transition TTC de dimension (HRCxHRC) et de format np.matrix
#     - de la PD projetée selon un scenario baseline de format dict={'annee':pd}
#     - de la PD projetée selon un scenario adverse de format dict={'annee':pd}
#     - de l'EAD par HRC (moyenne ou PIT au choix) de dimension (HRCx1) et de format np.array

class matrice_HRC:
    def __init__(self,matrice_ttc,pd_base,pd_adv,ead_pit):
        self.hrc=list(matrice_ttc.columns)
        self.m_ttc=np.matrix(matrice_ttc.iloc[:,1:])
        self.pd_proj={'base':pd_base,'adv':pd_adv}
        self.ead_pit=ead_pit
        self.pd_ttc=float(np.average(self.m_ttc[:,-1],0,self.ead_pit))
        self.corr=self.correlation_baloise()
        self.horizon=list(pd_base.keys())
        
        self.z={'base':self.z_macro('base'),'adv':self.z_macro('adv')}
        self.zd={'base':dict(zip(self.horizon,self.z['base'])),
                 'adv':dict(zip(self.horizon,self.z['adv']))}
        
        self.projection=self.set_projection()
        
    def correlation_baloise(self):
        a=0.03*(1-math.exp(-35*self.pd_ttc))/(1-math.exp(-35))
        b=0.16*(1-(1-math.exp(-35*self.pd_ttc))/(1-math.exp(-35)))
        return (a+b)
    
    def z_macro(self,scenario='base'):
        a=math.sqrt(1-self.corr)*norm.ppf(np.array(list(self.pd_proj[scenario].values())))
        b=norm.ppf(self.pd_ttc)
        return ((a-b)/math.sqrt(self.corr))
    
    #Calcul du vecteur de defaut avec en entrée
    #la PD TTC par CHR (CHR,1)
    #la correlation (1,1)
    #le facteur de risque systematique de l'année concernée (1,1)
    def merton_default(self,z):
        a=norm.ppf(self.m_ttc[:-1,-1])+math.sqrt(self.corr)*z
        b=math.sqrt(1-self.corr)
        return norm.cdf(a/b)
    
    #Construction de la matrice de transition projetée avec en entrée
    #Matrice de transition TTC (CHR,CHR)
    #le facteur de correlation (1,1)
    #le facteur de risque systématique (1,1)
    def transition_matrix_merton(self,z):
        nb_rating=self.m_ttc.shape[0]
        tm=np.matrix(np.zeros((nb_rating,nb_rating)))

        tm[:-1,-1]=self.merton_default(z)
        tm[nb_rating-1,nb_rating-1]=1
        for i in range(nb_rating):
            for j in reversed(range(nb_rating-1)):
                if i == nb_rating-1:
                    tm[i,j]=0
                else:
                    tm[i,j]=self.transition_merton(i,j,z,tm)
        return tm
    
    #Pour une CHR d'une année donnée, calcul les taux de transition entre avec en entrée
    #i et j CHR de départ et CHR d'arrivée
    #la matrice de transition TTC (CHR,CHR)
    #le vecteur de risque systématique (1,1)
    #la matrice de transition PIT under construction (CHR,CHR)
    def transition_merton(self,i,j,z,tm):
        tm_ttc_cum=self.m_ttc[i,j:].sum()
        if tm_ttc_cum>=1:
            temp=1-tm[i,j+1:].sum()
        else:
            a=(norm.ppf(tm_ttc_cum)+math.sqrt(self.corr)*z)/math.sqrt(1-self.corr)
            temp=norm.cdf(a)-tm[i,j+1:].sum()
        return temp
    
    #Construit un dictionnaire par scenario et par année avec les matrices projetées
    def set_projection(self):
        temp={}
        for i in self.pd_proj:
            scenario={}
            for j in self.pd_proj[i]:
                scenario[j]=self.transition_matrix_merton(self.zd[i][j])
            temp[i]=scenario
        return temp
    
    def get_projection(self):
        return self.projection

In [138]:
matrice_ttc=pd.DataFrame({'depuis':['a','b','c','d'],
                          'a':[0.8,0.7,0.6,0.4],
                          'b':[0.17,0.255,0.330,0.3],
                          'c':[0.03,0.045,0.07,0.2],
                          'd':[0.03,0.045,0.07,0.1]})
pd_base={'2019':0.05,'2020':0.055,'2021':0.06}
pd_adv={'2019':0.055,'2020':0.075,'2021':0.08}
ead_pit=np.array([300,600,100,300])

segment1=matrice_HRC(matrice_ttc,pd_base,pd_adv,ead_pit)

{'base': {'2019': matrix([[0.7806989 , 0.16569956, 0.02771816, 0.02588338],
          [0.66496763, 0.25299857, 0.04240711, 0.03962669],
          [0.53759615, 0.33201824, 0.06736914, 0.06301647],
          [0.        , 0.        , 0.        , 1.        ]]),
  '2020': matrix([[0.76666116, 0.17445734, 0.03006031, 0.0288212 ],
          [0.64780408, 0.26286072, 0.04555173, 0.04378347],
          [0.51902991, 0.34043306, 0.07153622, 0.06900081],
          [0.        , 0.        , 0.        , 1.        ]]),
  '2021': matrix([[0.7531626 , 0.18268399, 0.03235445, 0.03179896],
          [0.63155669, 0.27188947, 0.04859294, 0.0479609 ],
          [0.50171523, 0.34783123, 0.07549926, 0.07495428],
          [0.        , 0.        , 0.        , 1.        ]])},
 'adv': {'2019': matrix([[0.76666116, 0.17445734, 0.03006031, 0.0288212 ],
          [0.64780408, 0.26286072, 0.04555173, 0.04378347],
          [0.51902991, 0.34043306, 0.07153622, 0.06900081],
          [0.        , 0.        , 0.        ,